## Procesamiento de Lenguaje Natural

![Colegio Bourbaki](./Images/Bourbaki.png)

## Fast Text: Representación de palabras mediante subpalabras

Link de Interés: https://fasttext.cc/

FastText es un método de embedding de palabras desarrollado por **Facebook AI Research (FAIR)**. Es una extensión del modelo Word2Vec, pero introduce una mejora clave: en lugar de aprender vectores solo para palabras completas, representa cada palabra como un conjunto de n-gramas de caracteres.

- Representación mediante n-gramas

Por ejemplo, si tomamos la palabra “artificial” y definimos 
n=3
n=3:

Los n-gramas de caracteres serían:

<ar, art, rti, tif, ifi, fic, ici, cia, ial, al>

(A menudo se añaden los delimitadores de inicio y fin, como < y >, para preservar el contexto).

Así, la representación vectorial de la palabra se obtiene como la suma o promedio de los vectores de sus n-gramas.

- Ventajas sobre Word2Vec

Generalización a palabras desconocidas (OOV):
Si una palabra no se vio durante el entrenamiento, FastText puede descomponerla en sus subpalabras (n-gramas) y construir un vector aproximado, lo que Word2Vec no puede hacer.

Comprensión morfológica:
Al basarse en subpalabras, captura información sobre prefijos, sufijos y raíces, lo que mejora la representación de palabras con similitudes morfológicas (por ejemplo, “correr”, “corriendo”, “corredor”).

- Modelo de entrenamiento

El modelo subyacente es similar a Skip-gram o CBOW (como en Word2Vec), pero cada palabra se trata como la suma de sus n-gramas de caracteres.
Aunque internamente no modela explícitamente la estructura lingüística de la oración, utiliza una ventana deslizante de contexto sobre las palabras, como un modelo de bolsa de palabras (bag of words).

![fastText](./Images/cbow-sg.png)

**Aplicaciones prácticas**

FastText se puede aplicar a numerosos problemas relacionados con el lenguaje natural:

- Corrección ortográfica y autocompletado.

- Sistemas de recomendación y búsqueda: mejores sugerencias de productos o consultas.

- Chatbots y atención al cliente: comprensión de entradas con errores tipográficos.

- Análisis de sentimientos y reseñas.

Se pueden usar conjuntos de datos como:

- Consultas de búsqueda de usuarios.

- Conversaciones de chatbots.

- Reseñas y valoraciones.

Estos modelos pueden mejorar la experiencia del cliente al ofrecer sugerencias más precisas, autocorrección, o resultados más relevantes.


FastText es un modelo de embedding basado en subpalabras (n-gramas) que mejora a Word2Vec al poder representar palabras raras o no vistas y captar mejor la morfología del idioma.
Gracias a esto, es especialmente útil en aplicaciones prácticas donde la entrada del usuario puede ser ruidosa o diversa.

---

### Corrector ortográfico con FastText

En este notebook construimos un **corrector ortográfico inteligente** combinando técnicas clásicas de NLP con **embeddings FastText**.

### ¿Qué hace el código?
1. **Tokenización del texto**  
   Dividimos las oraciones en palabras, espacios y signos de puntuación.  
   Solo intentamos corregir las palabras alfabéticas, manteniendo el resto intacto.

2. **Generación de candidatos (edits)**  
   Para cada palabra mal escrita, generamos posibles variantes aplicando operaciones de edición:
   - Eliminación de una letra  
   - Inserción de una letra  
   - Sustitución de una letra  
   - Transposición de dos letras seguidas  

   Además, consideramos los errores más comunes de teclado (QWERTY), donde confundir teclas adyacentes tiene menor costo.

3. **Distancia de edición ponderada**  
   Calculamos qué tan diferente es cada candidato de la palabra original, asignando costos menores a errores típicos de tipeo.

4. **Embeddings de FastText**  
   Usamos vectores de palabras (FastText) para comparar la similitud semántica entre la palabra mal escrita y cada candidato.  
   Esto permite corregir incluso palabras fuera del vocabulario (OOV) gracias a los sub-tokens de FastText.

5. **Ranking de candidatos**  
   Combinamos:
   - Similitud de embeddings  
   - Frecuencia de la palabra en el corpus  
   - Distancia de edición  

   El mejor candidato según esta puntuación se elige como corrección.

6. **Reconstrucción del texto**  
   Reemplazamos solo las palabras corregidas y dejamos intactos los espacios y la puntuación, devolviendo la oración completa.

---

En resumen: el sistema propone correcciones ortográficas basadas no solo en la cercanía de edición, sino también en la **probabilidad de uso real** (frecuencia) y la **cercanía semántica**

### Librerias

In [7]:
# Utilities
from __future__ import annotations
import math
import re
import string
from collections import Counter, defaultdict
from collections.abc import Iterable
from dataclasses import dataclass
import numpy as np

# Wordfreq for word frequencies
from wordfreq import zipf_frequency 

# Gensim for FastText
import gensim.downloader as api
from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
from gensim.models.fasttext import load_facebook_vectors

### Funciones de ayuda

In [8]:
# Regex for tokenization
TOKEN_RE = re.compile(r"([A-Za-z]+|[^A-Za-z\s]+|\s+)")

Usamos esa regex para separar el texto en tokens manteniendo todo, de manera que al corregir podamos reconstruir la oración sin perder espacios o símbolos.

In [9]:
def tokenize(text: str) -> list[str]:
    """Split into tokens: words, whitespace, punctuation blocks. Keep everything to reconstruct."""
    return TOKEN_RE.findall(text)

In [10]:
def is_word(tok: str) -> bool:
    '''Returns true if token is a word (only letters).'''
    return tok.isalpha()

Importamos todas las letras minúsculas del alfabeto inglés desde la librería estándar string.
Esto se usa para probar reemplazos e inserciones de letras en las palabras.

In [11]:
ALPHABET = string.ascii_lowercase
ALPHABET

'abcdefghijklmnopqrstuvwxyz'

Construimos un diccionario de adyacencia de teclado (layout QWERTY en inglés).

Ejemplo: la letra s tiene como adyacentes a, d, w, x.

Esto sirve para que errores como “s → a” o “s → d” cuesten menos en la distancia de edición (porque son errores de tipeo comunes).

In [12]:
# Simple keyboard adjacency map (US QWERTY). Extend as needed.
KEY_ADJ: dict[str, set[str]] = defaultdict(set)

_adj_rows = [
    "qwertyuiop",
    "asdfghjkl",
    "zxcvbnm",
]
for row in _adj_rows:
    for i, ch in enumerate(row):
        if i > 0:
            KEY_ADJ[ch].add(row[i-1])
        if i < len(row)-1:
            KEY_ADJ[ch].add(row[i+1])

Generamos todas las palabras que están a una edición de distancia de la original:

**deletes** → eliminar una letra.

**transposes** → intercambiar dos letras consecutivas.

**replaces** → reemplazar una letra por otra del alfabeto.

**inserts** → insertar una letra extra en cualquier posición.

Estas son las candidatas para corrección.

In [13]:
def edits(word: str) -> set[str]:
    word = word.lower()
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in ALPHABET]
    inserts  = [L + c + R for L, R in splits for c in ALPHABET]
    return set(deletes + transposes + replaces + inserts)

Implementamos una variante del algoritmo de Damerau-Levenshtein para calcular la distancia entre dos palabras:

Inserción = costo 1

Eliminación = costo 1

Sustitución = costo 1 (pero si la sustitución es entre teclas adyacentes → costo 0.6)

Transposición = costo 0.8

Así modelamos errores más realistas de escritura.

In [14]:
def weighted_edit_distance(src: str, dst: str) -> float:
    """Damerau-Levenshtein style with lower cost for adjacent-key substitutions."""
    src, dst = src.lower(), dst.lower()
    n, m = len(src), len(dst)
    dp = np.zeros((n+1, m+1), dtype=float)
    for i in range(n+1):
        dp[i,0] = i
    for j in range(m+1):
        dp[0,j] = j
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost_sub = 0 if src[i-1]==dst[j-1] else (0.6 if dst[j-1] in KEY_ADJ[src[i-1]] else 1.0)
            dp[i,j] = min(
                dp[i-1,j] + 1,         # deletion
                dp[i, j-1] + 1,        # insertion
                dp[i-1,j-1] + cost_sub # substitution
            )
            if i>1 and j>1 and src[i-1]==dst[j-2] and src[i-2]==dst[j-1]:
                dp[i,j] = min(dp[i,j], dp[i-2,j-2] + 0.8)  # transposition
    return float(dp[n,m])

Vamos a definir un conjunto de parámetros ajustables para balancear los criterios de correción.

Por ejemplo, cuando evaluamos un candidato, el sistema calcula algo por el estilo:

score = sim_w * similitud + freq_w * log_frecuencia - dist_w * distancia

El candidato con mayor score es el que elegimos como corrección.

* sim_w → controla cuánto valoramos la similitud semántica entre la palabra original y el candidato, medida con FastText (cosine similarity).

* freq_w → controla cuánto valoramos que el candidato sea una palabra frecuente en el vocabulario.

* dist_w → controla cuánto penalizamos candidatos que están muy lejos de la palabra original en términos de operaciones de edición.

In [15]:
@dataclass
class RankWeights:
    sim_w: float = 2.0  # peso para la similitud de embeddings (coseno)
    freq_w: float = 1.2  # peso para la frecuencia de la palabra en el corpus
    dist_w: float = 1.0  # penalización por distancia de edición

Vamos a definir la clase principal del corrector ortográfico utilizando FastText. 
Esta clase incluirá métodos para entrenar el modelo, generar candidatos de corrección y seleccionar la mejor corrección basada en una puntuación compuesta, es decir,
la tokenización, la verificación de palabras conocidas, la generación de ediciones, el cálculo de similitud de embeddings y la distancia de edición ponderada, para entrenar, cargar, generar candidatos y corregir texto.

In [16]:
class SpellChecker:
    def __init__(
        self,
        model=None,  
        vocab: Counter | None = None,  
        weights: RankWeights | None = None, 
    ):
        self.model = model  # modelo FastText (entrenado o cargado)
        self.vocab = vocab or Counter()  # frecuencias de palabras
        self.weights = weights or RankWeights()  # pesos de ranking
        self._wordset: set[str] = (
            set()
        )  # conjunto rápido para saber si una palabra existe
        try:
            zipf_frequency,
            # escala ~0..7 (más alto = más frecuente)
            self.zipf_frequency = zipf_frequency
        except Exception:
            self.zipf_frequency = None

    # Model & vocab
    def train_model(
        self,
        corpus: Iterable[str],
        size: int = 100,
        window: int = 5,
        min_count: int = 1,
        epochs: int = 10,
    ) -> None:
        sentences = [re.findall(r"[A-Za-z]+", s.lower()) for s in corpus]
        self.model = FastText(vector_size=size, window=window, min_count=min_count)
        self.model.build_vocab(sentences)
        self.model.train(sentences, total_examples=len(sentences), epochs=epochs)
        self.vocab = Counter(w for sen in sentences for w in sen)
        self._wordset = set(self.vocab)

    def load_pretrained(self, path: str) -> None:
        if path.endswith(".bin"):
            try:
                self.model = KeyedVectors.load_word2vec_format(path, binary=True)
            except Exception:
                self.model = load_facebook_vectors(path)
        else:
            self.model = KeyedVectors.load_word2vec_format(path, binary=False)
        self._wordset = set(self.get_vocab_words())

    def set_vocab_from_corpus(self, corpus: Iterable[str]) -> None:
        words = [w.lower() for s in corpus for w in re.findall(r"[A-Za-z]+", s)]
        self.vocab = Counter(words)
        self._wordset = set(self.vocab)

    def get_vocab_words(self) -> Iterable[str]:
        m = self.model
        if hasattr(m, "key_to_index"):
            return list(m.key_to_index.keys())
        if hasattr(m, "wv") and hasattr(m.wv, "key_to_index"):
            return list(m.wv.key_to_index.keys())
        return list(self.vocab.keys())

    # Scoring
    def _freq_prior(self, w):
        # Prior por frecuencia de palabra:
        # - Si wordfreq: usa Zipf (0..7).
        # - Si no, usa log-frecuencia del vocab manual.
        if self.zipf_frequency is not None:
            return self.zipf_frequency(w.lower(), "en")  # cambiar "en" por otro idioma
        return math.log(self.vocab.get(w.lower(), 0) + 1.0)

    def _cosine(self, a, b):
        # Similitud coseno entre embeddings de palabras
        m = self.model
        if m is None:
            return 0.0
        try:
            if hasattr(m, "wv"):
                va, vb = m.wv[a], m.wv[b]
            else:
                va, vb = m[a], m[b]
            denom = (np.linalg.norm(va) * np.linalg.norm(vb))
            if denom == 0:
                return 0.0
            return float(np.dot(va, vb) / denom)
        except Exception:
            return 0.0

    def score(self, src, cand, prev=None, nxt=None):
        # Componentes
        d   = weighted_edit_distance(src, cand)
        sim = self._cosine(src.lower(), cand.lower())
        fr  = self._freq_prior(cand)

        # Contexto (promedio de similitud con vecinos alfabéticos si existen)
        ctx = 0.0
        cnt = 0
        if prev and prev.isalpha():
            ctx += self._cosine(cand.lower(), prev.lower()); cnt += 1
        if nxt and nxt.isalpha():
            ctx += self._cosine(cand.lower(), nxt.lower());  cnt += 1
        if cnt:
            ctx /= cnt

        # Pequeño boost si el candidato ya es igual (evita cambios innecesarios)
        ident_bonus = 0.15 if src.lower() == cand.lower() else 0.0

        # Cálculo final
        return ident_bonus + (2.5 * sim) + (1.4 * fr) + (1.0 * ctx) - (1.2 * d)

    # Candidates
    def known(self, words: Iterable[str]) -> set[str]:
        if self._wordset:
            return {w for w in words if w in self._wordset}
        return {w for w in words if self.vocab.get(w, 0) > 0}

    def candidates(self, word, max_edits=2, k_from_embeddings=200):
        wl = word.lower()
        cands = set()

        # a) vecinos por edición
        cands |= self.known({wl})
        e1 = self.known(edits(wl))
        cands |= e1
        if max_edits >= 2 and e1:
            for w1 in list(e1)[:300]:  # limitar expansión para no explotar el espacio
                cands |= self.known(edits(w1))

        # b) vecinos por embeddings (si hay modelo)
        try:
            m = self.model.wv if hasattr(self.model, "wv") else self.model
            if hasattr(m, "most_similar"):
                for w, _ in m.most_similar(wl, topn=k_from_embeddings):
                    cands.add(w)
        except Exception:
            pass
        if not cands:
            cands.add(wl)

        # c) filtro suave por frecuencia (descarta palabras muy raras)
        #    umbral ajustable: si existe wordfreq, pedimos Zipf>=2.5; si no, al menos que aparezca en vocab
        filtered = set()
        for w in cands:
            if self.zipf_frequency is not None:
                if self.zipf_frequency(w, "en") >= 2.5:
                    filtered.add(w)
            else:
                if self.vocab.get(w, 0) > 0:
                    filtered.add(w)
        if filtered:
            cands = filtered
        return cands

    # Correction
    def correct_word(self, word, prev=None, nxt=None, max_edits=2):
        if len(word) <= 2:
            return word
        cands = self.candidates(word, max_edits=max_edits)
        best = max(cands, key=lambda c: self.score(word, c, prev=prev, nxt=nxt))
        if word.istitle():
            return best.title()
        if word.isupper():
            return best.upper()
        return best

    def correct(self, text, max_edits=2):
        toks = tokenize(text)
        out = []
        # Para cada token palabra, miramos palabra previa/siguiente (alfabéticas) como contexto
        for i, tok in enumerate(toks):
            if tok.isalpha():
                prev = None
                nxt  = None
                # buscar prev
                j = i - 1
                while j >= 0:
                    if toks[j].isalpha(): prev = toks[j]; break
                    j -= 1
                # buscar next
                j = i + 1
                while j < len(toks):
                    if toks[j].isalpha(): nxt = toks[j]; break
                    j += 1
                out.append(self.correct_word(tok, prev=prev, nxt=nxt, max_edits=max_edits))
            else:
                out.append(tok)
        return "".join(out)

* Constructor __init__

Se inicializa el corrector con un modelo de embeddings, un vocabulario y los parámetros de peso.


* Sección “Model & vocab”

train_model → entrena un modelo FastText desde cero sobre un corpus dado.

load_pretrained → carga un modelo ya entrenado desde archivo (.bin o texto).

set_vocab_from_corpus → construye un contador de frecuencias a partir de un corpus.

get_vocab_words → obtiene la lista de palabras conocidas según el modelo o vocabulario.

Esto define la base de conocimiento del corrector.


* Sección “Scoring”

_cosine → calcula la similitud de coseno entre embeddings de dos palabras.

_log_freq → devuelve el logaritmo de la frecuencia de una palabra (más frecuente = más probable).

score → combina: bonus de identidad (si ya coincide la palabra), similitud de embeddings, frecuencia, penalización por distancia de edición.

Da un puntaje total para ordenar candidatos.

* Sección “Candidates”

known → filtra una lista de palabras dejando solo las que están en el vocabulario/modelo.

candidates → genera posibles correcciones:

Variantes por ediciones (borrar, insertar, sustituir, transponer).

Vecinos semánticos según embeddings (most_similar).

Si no encuentra nada, devuelve la palabra original.

Aquí obtenemos qué opciones existen para corregir una palabra.

* Sección “Correction”

correct_word → evalúa todos los candidatos de una palabra y se queda con el de mayor score. Respeta mayúsculas/minúsculas.

correct → tokeniza el texto completo, corrige solo las palabras alfabéticas, y reconstruye el texto final.

Esta es la función que usamos directamente para corregir frases enteras.

Vamos a implementar una función que nos permita construir un corrector. Tenemos 3 modos de uso:

* Modo pretrained (pretrained no es None)

Carga un modelo FastText ya entrenado (ej. cc.en.300.bin).
Si el vocabulario está vacío, se inicializa con una lista de hasta 200k palabras del modelo.

* Modo demo (demo=True)

Entrena un mini-modelo FastText sobre el TOY_CORPUS.
Esto permite probar el pipeline sin descargar nada.

* Modo fallback (ni pretrained ni demo)

Usa solo el vocabulario del TOY_CORPUS, sin embeddings.
Es el modo más básico, útil si no hay FastText disponible.

In [17]:
TOY_CORPUS = [
    "This is a simple sentence.",
    "Another simple example sentence.",
    "We write code to correct spelling errors.",
    "Fasttext uses subword embeddings.",
    "Spell checking benefits from language models",
]

In [18]:
def build_checker(pretrained=None, demo: bool = True) -> SpellChecker:
    """
    Construye un SpellChecker:
      - Si pretrained es un str -> se asume ruta a archivo .bin/.txt
      - Si pretrained es un objeto gensim (KeyedVectors/FastText) -> se usa directamente
      - Si demo=True -> entrena mini-modelo en TOY_CORPUS
      - Sino -> usa solo vocabulario de TOY_CORPUS (sin embeddings)
    """
    checker = SpellChecker(weights=RankWeights(sim_w=2.0, freq_w=1.2, dist_w=1.0))

    if pretrained is not None:
        # Caso 1: ruta a archivo
        if isinstance(pretrained, str):
            checker.load_pretrained(pretrained)
        else:
            # Caso 2: objeto de modelo gensim ya cargado (ej. api.load)
            checker.model = pretrained

        # inicializar vocabulario si está vacío
        if not checker.vocab:
            words = list(checker.get_vocab_words())[:200000]
            checker.vocab = Counter({w: 1 for w in words})
            checker._wordset = set(words)

    elif demo:
        checker.train_model(TOY_CORPUS, size=100, window=5, min_count=1, epochs=10)

    else:
        checker.vocab = Counter(w for s in TOY_CORPUS for w in s.split())
        checker._wordset = set(checker.vocab)

    return checker

Veamos cómo usarlo:

In [19]:
sample = "Ths is a smple sentnce with speling erors."

In [20]:
# Build the checker with a tiny demo FastText model (no external downloads)
checker = build_checker(pretrained=None, demo=True)
print("INPUT :", sample)
print("OUTPUT:", checker.correct(sample))

INPUT : Ths is a smple sentnce with speling erors.
OUTPUT: This is a simple sentence to spelling to.


In [21]:
model = api.load("fasttext-wiki-news-subwords-300")

In [22]:
checker = build_checker(pretrained=model, demo=False)
sample = "Ths is a smple sentnce with speling erors."
print("INPUT :", sample)
print("OUTPUT:", checker.correct(sample))

INPUT : Ths is a smple sentnce with speling erors.
OUTPUT: The is a simple sentence with spelling errors.


In [23]:
checker = build_checker(pretrained=None, demo=False)
sample = "Ths is a smple sentnce with speling erors."
print("INPUT :", sample)
print("OUTPUT:", checker.correct(sample))

INPUT : Ths is a smple sentnce with speling erors.
OUTPUT: Ths is a simple sentnce with spelling erors.


In [ ]:
import nltk

nltk.download("brown")

from nltk.corpus import brown

[nltk_data] Downloading package brown to /home/pdconte/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [25]:
corpus = [" ".join(sent) for sent in brown.sents()]
checker = build_checker(demo=False)
checker.train_model(corpus)
print(checker.correct("Ths is a smple sentnce with speling erors."))

The is a simple sentence with spelling errors.


In [26]:
examples = [
    "Ths is a smple sentnce with speling erors.",
    "Shee liks to read boks in the librery.",
    "We are lernig how to corect mispelled wrds.",
    "The quik borwn fox jmps ovr the lazi dog.",
    "Natrual langauge procesing is intersting.",
]

for s in examples:
    print("INPUT :", s)
    print("OUTPUT:", checker.correct(s))
    print("-" * 60)

INPUT : Ths is a smple sentnce with speling erors.
OUTPUT: The is a simple sentence with spelling errors.
------------------------------------------------------------
INPUT : Shee liks to read boks in the librery.
OUTPUT: She like to read books in the library.
------------------------------------------------------------
INPUT : We are lernig how to corect mispelled wrds.
OUTPUT: We are lerner how to correct spelled words.
------------------------------------------------------------
INPUT : The quik borwn fox jmps ovr the lazi dog.
OUTPUT: The quick born for jumps over the lazy dog.
------------------------------------------------------------
INPUT : Natrual langauge procesing is intersting.
OUTPUT: Natural language processing is interesting.
------------------------------------------------------------


In [27]:
checker.correct_word("speling")


'spelling'

In [28]:
checker.correct_word("smple", prev="a", nxt="sentence")

'simple'

In [29]:
cands = checker.candidates("speling")
print("Candidatos para 'speling':", sorted(list(cands)))

Candidatos para 'speling': ['ailing', 'baffling', 'boiling', 'bombing', 'briefing', 'bubbling', 'bursting', 'calming', 'chafing', 'charging', 'charming', 'chatting', 'chugging', 'circling', 'clawing', 'cling', 'copying', 'creaking', 'cringing', 'crippling', 'crumbling', 'curbing', 'curling', 'curving', 'dabbling', 'dangling', 'debunking', 'deeming', 'despairing', 'dialing', 'digging', 'dreaming', 'dressing', 'drugging', 'dueling', 'dwindling', 'embezzling', 'embodying', 'emitting', 'fairing', 'fawning', 'fencing', 'filing', 'fleming', 'fooling', 'fumbling', 'grappling', 'grieving', 'growling', 'humming', 'hurtling', 'hustling', 'idling', 'inkling', 'juggling', 'kipling', 'labeling', 'lingering', 'linking', 'meddling', 'mingling', 'modeling', 'mulling', 'mumbling', 'muttering', 'nailing', 'naming', 'paging', 'paneling', 'patting', 'peeling', 'piling', 'pleading', 'polling', 'pooling', 'portraying', 'purging', 'puzzling', 'rambling', 'reeling', 'ribbing', 'ridiculing', 'ruling', 'rumblin

In [30]:
word = "speling"
for cand in checker.candidates(word):
    print(cand, "→", checker.score(word, cand))

digging → 3.7052169713974
trucking → 1.6046089363098144
paneling → 2.954586882591247
spelling → 6.863473677635192
wrapping → 1.9276108903884888
sealing → 4.942210045576095
baffling → 2.500096553087234
stinking → 2.380250285625457
linking → 3.6681769719123833
scrambling → 1.43938517165184
briefing → 2.8762612447738647
screaming → 3.7351446533203125
ruling → 4.867081147193909
hurtling → 1.7070206418037408
pleading → 2.82184118103981
signaling → 2.760330406427383
reeling → 4.3842892656326296
stirling → 4.200774427175522
despairing → 0.12614150714874217
grappling → 2.624631917953491
naming → 3.0762742764949795
lingering → 1.2601444263458248
purging → 2.2320652267932886
inkling → 2.903462163209915
spiraling → 3.35547372508049
surging → 3.5404468657970423
curving → 2.407335652351379
ribbing → 1.1845391852855682
fooling → 3.9569705739021295
sizzling → 3.218874657869339
calming → 2.3263598227500912
dressing → 3.970941685914992
portraying → -0.33065012383460957
sniffing → 2.1781785161495204
bur

In [31]:
text = "Ths is a smple sentnce."
print(tokenize(text))

['Ths', ' ', 'is', ' ', 'a', ' ', 'smple', ' ', 'sentnce', '.']


In [32]:
print("Tamaño de vocabulario:", len(checker.vocab))
print("Palabras más frecuentes:", checker.vocab.most_common(10))

Tamaño de vocabulario: 41433
Palabras más frecuentes: [('the', 70003), ('of', 36473), ('and', 28935), ('to', 26247), ('a', 23517), ('in', 21422), ('that', 10789), ('is', 10109), ('was', 9815), ('he', 9801)]


### Referencias

- Word Representations: https://fasttext.cc/docs/en/unsupervised-tutorial.html

- FastText GitHub: https://github.com/facebookresearch/fastText/

- Building a spelling correction/word suggestion module: https://github.com/Svantevith/spelling-correction-module-using-fasttext/blob/master/Spelling_corrector_word_suggestion_module_using_fastText.ipynb

- system_transcription_error_detection_and_correction: https://github.com/kmariael/system_transcription_error_detection_and_correction

- Misspelling-Correction-with-fastText-sequence-labeling: https://github.com/leechehao/Misspelling-Correction-with-fastText-sequence-labeling

- Enriching Word Vectors with Subword Information: https://arxiv.org/pdf/1607.04606

- Bag of Tricks for Efficient Text Classification: https://arxiv.org/abs/1607.01759

- FastText.zip: Compressing text classification models: https://arxiv.org/abs/1612.03651